<a href="https://colab.research.google.com/github/zainali60/PIAIC-Q3/blob/main/Disasater_Tweet_Classification_bert_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#Loading libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import plotly.graph_objects as go
import plotly.figure_factory as ff

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

import re
import string

In [43]:
train = pd.read_csv("/content/train.csv",encoding='latin')
test = pd.read_csv("/content/test.csv",encoding='latin')
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [45]:
df = train.drop(['keyword', 'location'], axis=1)
df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [47]:
df.isnull().sum()

id        0
text      0
target    0
dtype: int64

In [48]:
len(df[df['target']==1])

3271

In [49]:
len(df[df['target']==0])

4342

In [55]:
a= df.target.value_counts()

In [56]:
fig = px.bar(a, x=[0,1], y='target',color=[0,1])
fig.show()

In [57]:
def cleaner(tweet):
    
    # remove urls
    tweet = re.sub(r'http\S+', ' ', tweet)
    
    # remove html tags
    tweet = re.sub(r'<.*?>',' ', tweet)
    
    # remove digits
    tweet = re.sub(r'\d+',' ', tweet)
    
    # remove hashtags
    tweet = re.sub(r'#\w+',' ', tweet)
    
    # remove mentions
    tweet = re.sub(r'@\w+',' ', tweet)
    
    #removing stop words
    tweet = tweet.split()
    tweet = " ".join([word for word in tweet if not word in stop_words])
    return  tweet

In [62]:
df['text'] = df['text'].apply(cleaner)
df.head()

,id,text,target
0,1,Our Deeds Reason May ALLAH Forgive us,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked 'shelter place' notified o...,1
3,6,", people receive evacuation orders California",1
4,7,Just got sent photo Ruby smoke pours school,1


In [66]:
df['text']=df['text'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation))))

In [67]:
df.text[5]

'Update  California Hwy closed directions due Lake County fire '

In [68]:
df.text[7612]

'The Latest More Homes Razed Northern California Wildfire  ABC News'

In [83]:
!pip install transformers --q
!pip install datasets --q

     |████████████████████████████████| 306 kB 4.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 132 kB 48.0 MB/s 
     |████████████████████████████████| 243 kB 49.0 MB/s 
     |████████████████████████████████| 271 kB 46.3 MB/s 
     |████████████████████████████████| 192 kB 47.5 MB/s 
     |████████████████████████████████| 160 kB 49.1 MB/s 


In [84]:
from datasets import Dataset

In [71]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, default_data_collator, Trainer, set_seed
set_seed(42)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [72]:
maxx, minn = 0, 1000000
for index, row in df.iterrows():
    if len(row['text'].split())>maxx:
        maxx=len(row['text'].split())
    if len(row['text'].split())<minn:
        minn=len(row['text'].split())
maxx, minn

(25, 0)

In [73]:
max_length=64
df['input_ids']=df['text'].apply(lambda x: tokenizer(x, max_length=max_length, padding='max_length')['input_ids'])
df.head(3)

,id,text,target,input_ids
0,1,Our Deeds Reason May ALLAH Forgive us,1,"[101, 2256, 15616, 3114, 2089, 16455, 9641, 21..."
1,4,Forest fire near La Ronge Sask Canada,1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187..."
2,5,All residents asked shelter place notified off...,1,"[101, 2035, 3901, 2356, 7713, 2173, 19488, 373..."


In [74]:
df.rename(columns={'target':'labels'}, inplace=True)
df.head()

,id,text,labels,input_ids
0,1,Our Deeds Reason May ALLAH Forgive us,1,"[101, 2256, 15616, 3114, 2089, 16455, 9641, 21..."
1,4,Forest fire near La Ronge Sask Canada,1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187..."
2,5,All residents asked shelter place notified off...,1,"[101, 2035, 3901, 2356, 7713, 2173, 19488, 373..."
3,6,people receive evacuation orders California,1,"[101, 2111, 4374, 13982, 4449, 2662, 102, 0, 0..."
4,7,Just got sent photo Ruby smoke pours school,1,"[101, 2074, 2288, 2741, 6302, 10090, 5610, 103..."


In [75]:
df = df[['input_ids', 'labels']]
df.head()

,input_ids,labels
0,"[101, 2256, 15616, 3114, 2089, 16455, 9641, 21...",1
1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...",1
2,"[101, 2035, 3901, 2356, 7713, 2173, 19488, 373...",1
3,"[101, 2111, 4374, 13982, 4449, 2662, 102, 0, 0...",1
4,"[101, 2074, 2288, 2741, 6302, 10090, 5610, 103...",1


In [76]:
train_df = df[:-int(len(df)*0.01)].reset_index(drop=True)
test_df = df[-int(len(df)*0.01):].reset_index(drop=True)

In [80]:
train_df.dtypes

input_ids    object
labels        int64
dtype: object

In [81]:
len(train_df), len(test_df)

(7537, 76)

In [85]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [87]:
type(train_ds)

datasets.arrow_dataset.Dataset

In [88]:
batch_size = 16

args = TrainingArguments(
    'nlp-getting-started',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=1e-5,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)
data_collator = default_data_collator
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [89]:
trainer.train()

***** Running training *****
  Num examples = 7537
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 118


Epoch,Training Loss,Validation Loss
1,No log,0.212338
2,No log,0.189531


***** Running Evaluation *****
  Num examples = 76
  Batch size = 16
Saving model checkpoint to nlp-getting-started/checkpoint-59
Configuration saved in nlp-getting-started/checkpoint-59/config.json
Model weights saved in nlp-getting-started/checkpoint-59/pytorch_model.bin
tokenizer config file saved in nlp-getting-started/checkpoint-59/tokenizer_config.json
Special tokens file saved in nlp-getting-started/checkpoint-59/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 76
  Batch size = 16
Saving model checkpoint to nlp-getting-started/checkpoint-118
Configuration saved in nlp-getting-started/checkpoint-118/config.json
Model weights saved in nlp-getting-started/checkpoint-118/pytorch_model.bin
tokenizer config file saved in nlp-getting-started/checkpoint-118/tokenizer_config.json
Special tokens file saved in nlp-getting-started/checkpoint-118/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=118, training_loss=0.5059032440185547, metrics={'train_runtime': 1246.9075, 'train_samples_per_second': 12.089, 'train_steps_per_second': 0.095, 'total_flos': 1755991671350784.0, 'train_loss': 0.5059032440185547, 'epoch': 2.0})

In [90]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [91]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 76
  Batch size = 16


{'eval_accuracy': 0.9605263157894737,
 'eval_loss': 0.18953128159046173,
 'eval_runtime': 2.2406,
 'eval_samples_per_second': 33.92,
 'eval_steps_per_second': 2.232}

In [96]:
test.head(2)

,id,keyword,location,text
0,0,NaN,NaN,Just happened terrible car crash
1,2,NaN,NaN,"Heard different cities, stay safe everyone."


In [100]:
def cleaner(tweet):
    
    # remove urls
    tweet = re.sub(r'http\S+', ' ', tweet)
    
    # remove html tags
    tweet = re.sub(r'<.*?>',' ', tweet)
    
    # remove digits
    tweet = re.sub(r'\d+',' ', tweet)
    
    # remove hashtags
    tweet = re.sub(r'#\w+',' ', tweet)
    
    # remove mentions
    tweet = re.sub(r'@\w+',' ', tweet)
    
    #removing stop words
    tweet = tweet.split()
    tweet = " ".join([word for word in tweet if not word in stop_words])
    return  tweet
test['text'] = test['text'].apply(cleaner)
test.head(2)

,id,keyword,location,text
0,0,NaN,NaN,Just happened terrible car crash
1,2,NaN,NaN,"Heard different cities, stay safe everyone."


In [101]:
test['text']=test['text'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation))))


In [102]:
test['input_ids']=test['text'].apply(lambda x: tokenizer(x, max_length=max_length, padding='max_length')['input_ids'])
test.head()

,id,keyword,location,text,input_ids
0,0,NaN,NaN,Just happened terrible car crash,"[101, 2074, 3047, 6659, 2482, 5823, 102, 0, 0,..."
1,2,NaN,NaN,Heard different cities stay safe everyone,"[101, 2657, 2367, 3655, 2994, 3647, 3071, 102,..."
2,3,NaN,NaN,forest fire spot pond geese fleeing across str...,"[101, 3224, 2543, 3962, 8644, 28519, 14070, 24..."
3,9,NaN,NaN,Apocalypse lighting,"[101, 16976, 7497, 102, 0, 0, 0, 0, 0, 0, 0, 0..."
4,11,NaN,NaN,Typhoon Soudelor kills China Taiwan,"[101, 15393, 2061, 12672, 10626, 8563, 2859, 6..."


In [103]:
df_submission=test[['input_ids']]
df_submission.head()

,input_ids
0,"[101, 2074, 3047, 6659, 2482, 5823, 102, 0, 0,..."
1,"[101, 2657, 2367, 3655, 2994, 3647, 3071, 102,..."
2,"[101, 3224, 2543, 3962, 8644, 28519, 14070, 24..."
3,"[101, 16976, 7497, 102, 0, 0, 0, 0, 0, 0, 0, 0..."
4,"[101, 15393, 2061, 12672, 10626, 8563, 2859, 6..."


In [104]:
output_ds = Dataset.from_pandas(df_submission)
output_ds

Dataset({
    features: ['input_ids'],
    num_rows: 3263
})

In [107]:
output_ds.features

{'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [108]:
outputs=trainer.predict(output_ds)

***** Running Prediction *****
  Num examples = 3263
  Batch size = 16


In [109]:
outputs.predictions.argmax(1)

array([1, 0, 1, ..., 1, 1, 0])